In [ ]:
import os

import pandas

In [ ]:
def process_scenario(root_folder, year, era5_year, region):
    df_current_scenario = pandas.read_csv(
        f"{root_folder}/{year}/{era5_year}/{region}", delimiter=";"
    )

    df_current_scenario["era5 source year"] = era5_year.split("_")[-1]
    df_current_scenario["year"] = year

    # Namibia bug
    df_current_scenario.loc[
        df_current_scenario["region_name"] == "Namibia", "region_code"
    ] = "NA"

    df_current_scenario.sort_values(["region_code", "time"], inplace=True)

    df_current_scenario["datetime"] = pandas.to_datetime(df_current_scenario["time"])
    df_current_scenario["hour_of_year"] = (
        df_current_scenario["datetime"].dt.dayofyear - 1
    ) * 24 + df_current_scenario["datetime"].dt.hour

    df_current_scenario.drop(columns=["time", "datetime", "region_name"], inplace=True)

    return df_current_scenario

In [ ]:
root_folder = "./comparison/GEGIS/ssp2-2.6/"
df_all_scenarios = pandas.DataFrame()

for year in os.listdir(root_folder):
    for era5_year in os.listdir(os.path.join(root_folder, year)):
        for region in os.listdir(os.path.join(root_folder, year, era5_year)):
            if region.endswith(".csv"):
                df_all_scenarios = pandas.concat(
                    [
                        df_all_scenarios,
                        process_scenario(root_folder, year, era5_year, region),
                    ]
                )

In [ ]:
df_all_scenarios.to_parquet("./comparison/GEGIS/ssp2-2.6_processed.parquet")

In [ ]:
pandas.read_parquet("./comparison/GEGIS/ssp2-2.6_processed.parquet")

In [ ]:
df_current_scenario.dtypes